# Differentiable Physics 

<h1> Demos:</h1>

* <h2>Neural networks can be very dumb sometimes (inspired by <a hrerf="http://physicsbaseddeeplearning.com/intro-teaser.html">Physics Based Deep Learning</a> book)</h2>

* <h2>Differentiable optics models: design</h2>

* <h2>Differentiable optics models: inverse problems</h2>



In [ ]:
import os

import numpy as np
from autograd import numpy as anp
from autograd import grad

import skimage
import skimage.io as sio
import matplotlib.pyplot as plt

from src.asm_prop import save_as_gif, run_train


In [ ]:
def y_parabola(mode=0, batch_size=128, stretch=1.):
    
    x = np.random.rand(batch_size, 1) * stretch
    
    if mode == 0:
        y = np.sign(np.random.randn(batch_size))[:, np.newaxis] * np.sqrt(x)
        
    elif mode == 1:
        y = -1.0 * np.sqrt(x)
        
    elif mode == 2:
        y = np.sqrt(x)
        
    return x, y

def forward_nn(x, layers, biases):
    
    #activation = lambda x: x * (x > 0) + x * 0.1 * (x < 0)
    #activation = lambda x: x**2
    activation = lambda x: anp.sin(x**2)
    #activation = lambda x: anp.tanh(x)
    
    for layer, bias in zip(layers, biases):
        
        x = activation(anp.matmul(x, layer)) + bias
        
    return x


def forward_nn_loss(x, y_target, layers, biases):
        
    prediction = forward_nn(x, layers, biases)
    
    loss = anp.mean(anp.abs(y_target - prediction)**2)
    
    return loss


    
def get_layers(dim_x=1, dim_h=32, dim_y=1, number_h=1):
    
    layers = []
    biases = []
    
    layers.append(30 / (dim_x * dim_h) * np.random.randn(dim_x, dim_h))
    biases.append(np.random.randn(dim_h))
    

    for ii in range(number_h):
        layers.append(30 / (dim_h**2) * np.random.randn(dim_h, dim_h))
        biases.append(np.random.randn(dim_h))

    layers.append(30 / (dim_h * dim_y) * np.random.randn(dim_h, dim_y))
    biases.append(np.random.randn(dim_y))
    
    return layers, biases
    

get_nn_grad = grad(forward_nn_loss, argnum=(2,3))


x, y = y_parabola(batch_size = 2048)

max_steps = 2500
lr = 3e-4

layers, biases = get_layers()

val_x, val_y = y_parabola(batch_size = 128)

pred_y = forward_nn(val_x, layers, biases)

plt.figure()

plt.scatter(val_x, val_y)
plt.scatter(val_x, pred_y)

plt.show()


for ii in range(max_steps):
    

    if ii % (max_steps // 10) == 0:
        
        loss = forward_nn_loss(x, y, layers, biases)
        
        print(f"loss at step {ii} = {loss:.4}")
    
    grad_layers, grad_biases = get_nn_grad(x, y, layers, biases)
    
    for params, grads in zip(layers, grad_layers):
        params -=  lr * grads
        
    for params, grads in zip(biases, grad_biases):
        params -=  lr * grads
        

val_x, val_y = y_parabola(batch_size = 128, stretch=3)

pred_y = forward_nn(val_x, layers, biases)

plt.figure()

plt.scatter(val_x, val_y)
plt.scatter(val_x, pred_y)

plt.show()



In [ ]:
def forward_dp_loss(x, layers, biases):
        
    prediction = forward_nn(x, layers, biases)
    
    loss = anp.mean(anp.abs(x - prediction**2)**2)
    
    return loss

get_dp_grad = grad(forward_dp_loss, argnum=(1,2))

layers, biases = get_layers()

val_x, val_y = y_parabola(batch_size = 128)

pred_y = forward_nn(val_x, layers, biases)

plt.figure()

plt.scatter(val_x, val_y)
plt.scatter(val_x, pred_y)

plt.show()


for ii in range(max_steps):
    

    if ii % (max_steps // 10) == 0:
        
        loss = forward_dp_loss(x, layers, biases)
        
        print(f"loss at step {ii} = {loss:.4}")
    
    grad_layers, grad_biases = get_dp_grad(x, layers, biases)
    
    for params, grads in zip(layers, grad_layers):
        params -=  lr * grads
        
    for params, grads in zip(biases, grad_biases):
        params -=  lr * grads
        

val_x, val_y = y_parabola(batch_size = 128, stretch=3)

pred_y = forward_nn(val_x, layers, biases)

plt.figure()

plt.scatter(val_x, val_y)
plt.scatter(val_x, pred_y)

plt.show()


In [ ]:
image_url = input("Enter an image url (please):")

image_ext = os.path.splitext(image_url)[-1]


In [ ]:
if os.path.exists("./assets"):
    os.system(f"wget {image_url} -O ./assets/temp{image_ext}")
else:
    os.mkdir("./assets")
    os.system(f"wget {image_url} -O ./assets/temp{image_ext}")

my_image = sio.imread("./assets/temp.png")

if len(my_image.shape) > 2:
    my_image = my_image.mean(axis=-1)
    
plt.figure(figsize=(6,6))
plt.imshow(my_image, cmap="plasma")
plt.show()

os.system("rm ./assets/*gif")
images, losses, tgt_img = run_train(my_image)

plt.figure()
plt.plot(losses, lw=5)
plt.show()

In [ ]:
save_as_gif(images, filename="temp_0.gif")
sio.imsave("assets/tgt_0.png", tgt_img)

# Optimization Animation
<img src="assets/temp_0.gif" width=40%>

# Target Image
<img src="assets/tgt_0.png" width=40%>


## Optimization Animation
<img src="assets/temp1.gif" width=40%>

## Target Image
<img src="assets/tgt.png" width=40%>


In [ ]:
import autograd.numpy as np
from autograd import grad
import matplotlib.pyplot as plt
import time

import skimage
import skimage.io as sio
import skimage.transform

from PIL import Image

def asm_prop(wavefront, length=32.e-3, wavelength=550.e-9, distance=10.e-3):
        
    if len(wavefront.shape) == 2:
        dim_x, dim_y = wavefront.shape
    elif len(wavefront.shape) == 3:
        number_samples, dim_x, dim_y = wavefront.shape
    else:
        print("only 2D wavefronts or array of 2D wavefronts supported")

    assert dim_x == dim_y, "wavefront should be square"
    px = length / dim_x

    l2 = (1/wavelength)**2
    
    fx = np.linspace(-1/(2*px), 1/(2*px) - 1/(dim_x*px), dim_x)
    fxx, fyy = np.meshgrid(fx,fx)

    q = l2 - fxx**2 - fyy**2
    q[q<0] = 0.0

    h = np.fft.fftshift(np.exp(1.j * 2 * np.pi * distance * np.sqrt(q)))
    
    fd_wavefront = np.fft.fft2(np.fft.fftshift(wavefront)) 
    if len(wavefront.shape) == 3:
        fd_new_wavefront = h[np.newaxis,:,:] * fd_wavefront
        new_wavefront=np.fft.ifftshift(np.fft.ifft2(fd_new_wavefront))[:,:dim_x,:dim_x]
    else:
        fd_new_wavefront = h * fd_wavefront
        new_wavefront = np.fft.ifftshift(np.fft.ifft2(fd_new_wavefront))[:dim_x,:dim_x]


    return new_wavefront

def onn_layer(wavefront, phase_objects, d=100.e-3):

    for ii in range(len(phase_objects)):
        wavefront = asm_prop(wavefront * phase_objects[ii], distance=d)

    return wavefront

def get_loss(wavefront, y_tgt, phase_objects, d=100.e-3):

    img = np.abs(onn_layer(wavefront, phase_objects, d=d))**2
    mse_loss = np.mean( (img - y_tgt)**2 + np.abs(img-y_tgt) )

    return mse_loss

get_grad = grad(get_loss, argnum=2)

def save_as_gif(np_array, filename="my_gif.gif", my_cmap=None):
    
    assert (len(np_array.shape) == 3), "expected n by h by w array"
    
    if my_cmap == None:
        my_cmap = plt.get_cmap("magma")
        
    dim_x, dim_y = np_array.shape[-2], np_array.shape[-1]
    
    im = Image.fromarray((my_cmap(np_array[0])*255).astype("uint8"), "RGBA")

    im.save(f"assets/{filename}", save_all=True, duration=3*np_array.shape[0], loop=0, \
            append_images=[Image.fromarray((my_cmap(img)*255).astype("uint8"), "RGBA") for img in np_array[1:]])

def run_train(tgt_img, zero_pad=True):

    dim = 128
    side_length = 32.e-3
    aperture = 8.e-3
    wavelength = 550.e-9
    k0 = 2*np.pi / wavelength
    dist = 50.e-3

    if zero_pad:
        tgt_img = np.pad(tgt_img, (tgt_img.shape[0], tgt_img.shape[1]))
    # resize target image
    tgt_img = skimage.transform.resize(tgt_img, (dim, dim))
    
    px = side_length / dim

    x = np.linspace(-side_length/2, side_length/2-px, dim)

    xx, yy = np.meshgrid(x,x)
    rr = np.sqrt(xx**2 + yy**2)

    wavefront = np.zeros((dim,dim)) * np.exp(1.j*k0*0.0)
    wavefront[rr <= aperture] = 1.0

    #tgt_img = sio.imread("./smiley.png")[:,:,0]
    
    y_tgt = 1.0 * tgt_img / np.max(tgt_img)

    lr = 1e-3
    phase_objects = [np.exp(1.j * np.zeros((128,128)) ) \
            for aa in range(32)]
    losses = []

    training_arrays = []
    smooth_slope = 0.0
    
    for step in range(1024):


        my_grad = get_grad(wavefront, y_tgt, phase_objects, d=dist)

        for params, grads in zip(phase_objects, my_grad):
            params -=  lr * np.exp( -1.j * np.angle(grads))

        loss = get_loss(wavefront, y_tgt, phase_objects,d=dist)
        losses.append(loss)
        img = np.abs(onn_layer(wavefront, phase_objects))**2
        
        if step % 16 == 0:
            print("loss at step {} = {:.2e}, lr={:.3e}".format(step, loss, lr))
        
        training_arrays.append(img/2.0)
        
        if len(losses) > 1:
            smooth_slope = 0.99 * smooth_slope + 0.01 * (losses[-2] - losses[-1])
        
        if smooth_slope < 0.0:
            print("stopping training")
            break
    
    return np.array(training_arrays), losses, tgt_img